<h1> Facility Allocation Problem </h1>

<h3> Problem description </h3>

A company wants to set up a new European distribution network. There are *N* possible locations for a distribution center - **DC**. Fixed costs per year for opening a distribution center at location *i* (with a certain capacity **S(i)** in pallets) are **F(i)**.  The total demand per year (in pallets) for each delivery address - **DA** - *j* is **D(j)**. The transport costs (in EURO) for one pallet between the DCs and delivery addresses are **c(i,j)**.


**The question is: which DCs will be opened?**

We will use the **GUROBI** framework to solve this linear problem. After acquiring and installing a licence 
we can import it into our notebook.

In [1]:
from gurobipy import *

# Get some additional dependencies
import pandas as pd
import numpy as np

Lets start by denoting the list of our *N* distribution centers as well as the *M* delivery addresses.

**Note:** 
Intuitively the distance between a city and itself should be zero. However that would result to a matrix with 0s in its diagonal and therefore the determinant cannot be computed. This seems to yield some issues
with the LP solver, maybe GUROBI uses matrix inversion at some point? We will never know since its closed source. Anyway i just set these distances to 1 instead and things seem to work fine.

In [2]:
# Read the dataset
data = pd.read_csv('../data.csv')
data.replace('-', 1, inplace = True)
city_names = list(data)[1:]

# Denote model variables
N = len(data)
M = len(data)
DC = [i for i in range(0, N)]
DA = [i for i in range(0, M)]

# Hold a mapping from indexes to city names for showing results later
cities = dict(zip(DC, city_names))

Lets also denote the capacity **S** of every DC, and the demand **D** on every DA in pallets:

In [3]:
# Hyper parameters - will never change in this assignment
HUGE_CONSTANT = 1000000000000
FIXED_DEMAND = 100

S = HUGE_CONSTANT * np.ones(N)
D = FIXED_DEMAND * np.ones(M)

Finally, lets denote the given transportation cost per unit of pallets between DCs and DAs:

In [4]:
# Transform the pandas dataframe to a GUROBI aware dictionary
arr = data.as_matrix()[:,1:]
c = dict(((i,j), arr[i][j]) for i in range(len(arr)) for j in range(len(arr[0])))

arcs = list(c.keys())

Now that we have initiallized our inputs we are ready to construct our model.
Note that we include two constraints on our flow variable *x* directly into its declaration:

In [5]:
m = Model('facility_allocation')

x = {}
for i,j in arcs:
    x[i,j] = m.addVar(lb = 0,
                      vtype = GRB.CONTINUOUS, name='flow_%s_%s' % (i, j))


We also need the set of binary variables **Z**, where **z[i]** is True when DC *i* is open.
This condition can be expressed using the constraint x[i,j] <= M * z[i] where M is a sufficiently big value.

In [6]:
z = {}
for i in DC:
    z[i] = m.addVar(0.0, 1.0, 1.0, GRB.BINARY, 'isOpen_%s' % i)
    
m.addConstrs((quicksum(x[i,j] for j in DA) <= HUGE_CONSTANT * z[i]) for i in DC)
m.update()


Lets also apply additional constraints inferred from the problem statement. Namely, 

**The total amount sent to each DA should satisfy its demand**

In [7]:
m.addConstrs((quicksum(x[i,j] for i in DC) >= D[j]) for j in DA)
m.update()

**The total amount sent from each DC should be at most equal to its capacity**

In [8]:
m.addConstrs((quicksum(x[i,j] for j in DA) <= S[i]) for i in DC)
m.update()

At this point we are ready to define our objective function: The total cost.
This will obviously be equal to the sum of fixed and transportation costs.

In [9]:
def optimize(fixed_cost = 0):
    F = fixed_cost * np.ones(N)
    obj = LinExpr(0.0);
    obj.addTerms((c[i,j] for i in DC for j in DA), (x[i,j] for i in DC for j in DA))
    obj.addTerms((F[i] for i in DC), (z[i] for i in DC))
    m.setObjective(obj, GRB.MINIMIZE)
    m.setParam('OutputFlag', 0)
    m.update()
    m.optimize()

In [13]:
opened = set()
if m.status == GRB.Status.OPTIMAL:
    solution = m.getAttr("x", x)
    for i,j in arcs:
        if solution[i,j] > 0:
            opened.add(city_names[i])
            print('%s -> %s: %g' % (city_names[i], city_names[j], solution[i,j]))
            
#print("\n\n" + str(len(opened)) + " DCs were opened")

Our model is now complete. Lets define a macro parameter tuning function.
This should read a value for fixed costs, optimize the model and report
the optimal number of DCs for that value. 

In [14]:
def optimal_DC_num(fixed_cost = 100000):
    optimize(fixed_cost)
    if m.status == GRB.Status.OPTIMAL:
        opened = set()
        solution = m.getAttr("x", x)
        for i,j in arcs:
            if solution[i,j] > 0:
                opened.add(city_names[i])
        return len(opened)




**For which values of F we have that 1 DC is optimal? Which DC is then open? Same questions for 2, 3, 4, 5, 6, 7  DCs open.**

There is no easy way to automate the search since the mapping from fixed costs to optimal number of DCs is a blackbox. Of course I could write a recursive search (binary) but this would probably be out of scope. I therefore choose to manually search the fixed costs space using binary search. In order to accerelate this procedure we can try some points within a reasonable range of costs and get an idea of the
resulting DCs. We can then use that insight to start from a close range of costs at each iteration.

In [15]:
# Searching for 1 DC optimal

print(optimal_DC_num(1000000)) # output is 1
print(optimal_DC_num(500000)) # output is 7
print(optimal_DC_num(750000)) # output is 1
print(optimal_DC_num(600000)) # output is 2
print(optimal_DC_num(700000)) # output is 2
print(optimal_DC_num(730000)) # output is 1
print(optimal_DC_num(720000)) # output is 2
# Aha! turning point lies between 720 and 730k! This is accurate enough right...?


1
7
1
2
2
1
2


Using the approximation above we can see that **1 DC is opened for fixed costs higher than 730,000 euros.**

In that case, the DC is opened at **Strasbourg**. Some testing easily shows that this city has the minimum sum
of distances from all other cities in the dataset, therefore the result seems reasonable

**Using the same method in combination with our insight we get the following mapping:**
* 1 DC is optimal for fixed costs higher than 730,000 **[Strasburg]**
* 2 DCs are optimal for fixed costs between 330,000 and 730,000 **[Turin, Antwerp]**
* 3 DCs are optimal for fixed costs between 255,000 and 330,000 **[Milan, Antwerp, Madrid]**
* 4 DCs are optimal for fixed costs between 221,000 and 255,000 **[Milan, Antwerp, Madrid, Munich]**
* 5 DCs are optimal for fixed costs between 150,000 and 220,000 **[...]**
* 6 DCs are optimal for fixed costs between 130,000 and 150,000 **[...]**
* 7 DCs are optimal for fixed costs between 100,000 and 130,000 **[...]**

In [13]:
# Where should the DC be opened?
optimize(500000)

if m.status == GRB.Status.OPTIMAL:
    opened = set()
    solution = m.getAttr("x", x)
    for i,j in arcs:
        if solution[i,j] > 0:
            opened.add(city_names[i])
            print('%s -> %s: %g' % (city_names[i], city_names[j], solution[i,j]))
            
print("\n\n" + str(opened) + " DCs were opened")
        

Turin -> Venice: 100
Turin -> Lyon: 100
Antwerp -> Brussels: 100
Turin -> Madrid: 100
Antwerp -> Lisbon: 100
Turin -> Geneva: 100
Turin -> Bern: 100
Antwerp -> Cologne: 100
Antwerp -> Antwerp: 100
Turin -> Munich: 100
Antwerp -> Paris: 100
Turin -> Turin: 100
Antwerp -> The Hague: 100
Antwerp -> Frankfurt: 100
Turin -> Nice: 100
Antwerp -> Strasbourg: 100
Turin -> Barcelona: 100
Antwerp -> Le Havre: 100
Antwerp -> Amsterdam: 100
Turin -> Milan: 100
Antwerp -> Luxembourg: 100
Antwerp -> Edinburgh: 100
Turin -> Naples: 100
Antwerp -> Berlin: 100
Antwerp -> Rotterdam: 100
Turin -> Vienna: 100
Turin -> Zurich: 100
Turin -> Athens: 100
Antwerp -> Hamburg: 100
Turin -> Rome: 100
Antwerp -> Calais: 100
Turin -> Marseille: 100
Antwerp -> London: 100
Turin -> Genoa: 100
Antwerp -> Copenhagen: 100
Turin -> Stuttgart: 100
Antwerp -> Prague: 100


{'Turin', 'Antwerp'} DCs were opened
